In [1]:
# Importing libraries and packages:

from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn import metrics
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

Question A:  Read the dataset file “Cancer.csv” (from github using the following command), and assign it to a Pandas DataFrame

In [2]:
# A. Read the dataset file “Cancer.csv” (from github using the following command), and assign it to a Pandas DataFrame

cancer_df = pd.read_csv("https://github.com/mpourhoma/CS4661/raw/master/Cancer.csv")

cancer_df.head()

,Clump_Thickness,Uniformity_of_Cell_Size,Uniformity_of_Cell_Shape,Marginal_Adhesion,Single_Epithelial_Cell_Size,Bare_Nuclei,Bland_Chromatin,Normal_Nucleoli,Mitoses,Malignant_Cancer
0,5,1,1,1,2,1,3,1,1,0
1,5,4,4,5,7,10,3,2,1,0
2,3,1,1,1,2,2,3,1,1,0
3,6,8,8,1,3,4,3,7,1,0
4,4,1,1,3,2,1,3,1,1,0


Question B: Use sklearn functions to split the dataset into testing and training sets with the following parameters: test_size=0.35, random_state=3.

In [3]:
# create a python list of feature names that would like to pick from the dataset:
feature_cols = ['Clump_Thickness','Uniformity_of_Cell_Size','Uniformity_of_Cell_Shape',
                'Marginal_Adhesion','Single_Epithelial_Cell_Size','Bare_Nuclei',
                'Bland_Chromatin','Normal_Nucleoli','Mitoses']

# use the above list to select the features from the original DataFrame
X = cancer_df[feature_cols] 

# select a Series of labels (the last column) from the DataFrame
y = cancer_df['Malignant_Cancer']

# print the first 5 rows
print(X.head())
print(y.head())

   Clump_Thickness  Uniformity_of_Cell_Size  Uniformity_of_Cell_Shape  \
0                5                        1                         1   
1                5                        4                         4   
2                3                        1                         1   
3                6                        8                         8   
4                4                        1                         1   

   Marginal_Adhesion  Single_Epithelial_Cell_Size  Bare_Nuclei  \
0                  1                            2            1   
1                  5                            7           10   
2                  1                            2            2   
3                  1                            3            4   
4                  3                            2            1   

   Bland_Chromatin  Normal_Nucleoli  Mitoses  
0                3                1        1  
1                3                2        1  
2                3     

In [4]:
# Randomly splitting the original dataset into training set and testing set:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state=3)

# print the size of the traning set:
print(X_train.shape)
print(y_train.shape)

# print the size of the testing set:
print(X_test.shape)
print(y_test.shape)

(97, 9)
(97,)
(53, 9)
(53,)


Question C. Use “Decision Tree Classifier” to predict Cancer based on the training/testing datasets that you built in part (b). Then, calculate and report the accuracy of your classifier.

Use this command to define your tree: my_DecisionTree = DecisionTreeClassifier(random_state=3).

In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

my_DecisionTree = DecisionTreeClassifier(random_state=3)

my_DecisionTree.fit(X_train, y_train)

y_predict_dt = my_DecisionTree.predict(X_test)

score_dt = accuracy_score(y_test, y_predict_dt)

print("Decision Tree Accuracy: " + str(score_dt))

Decision Tree Accuracy: 0.8301886792452831


Question D: Now, we want to perform a new Ensemble Learning method called “Bagging” based on Voting on 19 decision tree classifiers.

In [6]:
#1. make a bootstarp sample of the original “Training” Dataset (build in part(b)) with the size of bootstarp_size = 0.8*
#2. Define and train a new base decision tree classifier on this dataset in each iteration:
#3. Perform prediction using “this base classifier” on the original “Testing” Dataset X_test (build in part(b)), and save the prediction results for all testing samples.

from sklearn.utils import resample
from sklearn.metrics import accuracy_score

predData = pd.DataFrame()
for i in range(19):
    bootstarp_size = int(0.8*len(X_train))
    i_X_train = resample(X_train, n_samples = bootstarp_size , random_state=i , replace = True)
    i_y_train = resample(y_train, n_samples = bootstarp_size , random_state=i , replace = True) 
    Base_DecisionTree = DecisionTreeClassifier(random_state=3)
    Base_DecisionTree.fit(i_X_train, i_y_train)
    predData[str(i)] = Base_DecisionTree.predict(X_test)

In [7]:
sumSample = pd.Series(predData.sum(axis=1))
p_Data = predData.assign(count = sumSample)
p_Accuracy = []

for i in range(len(predData)):
    if(sumSample[i]>= 10):
        p_Accuracy.append(1)
    else:
        p_Accuracy.append(0)
        
p_Data = predData.assign(final = pd.Series(p_Accuracy))

scoreP_Data = accuracy_score(y_test, p_Data['final'])
print("Prediction Final Score: " + str(scoreP_Data))



Prediction Final Score: 0.9056603773584906


In [8]:
from sklearn.ensemble import RandomForestClassifier
my_RandomForest = RandomForestClassifier(n_estimators = 19, bootstrap = True, random_state=3)
my_RandomForest.fit(X_train, y_train)
y_predict_rf = my_RandomForest.predict(X_test)
score_rf = accuracy_score(y_test, y_predict_rf)
print('Random Forest Accuracy Final Result:',score_rf)


Random Forest Accuracy Final Result: 0.9245283018867925
